In [ ]:
pip install keras-tuner -q

In [19]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
import keras_tuner
from sklearn.model_selection import train_test_split

## Preproccessing Hurricane Data

In [2]:
# Obtain Cyclone Data
df = pd.read_csv("../raw-data/hurricaneData.csv")

In [3]:
# Drop all attributes, expect Longitude, Latitude, Time and SID
simple_df = df.drop(['SEASON','NUMBER','BASIN','SUBBASIN','NAME','NATURE','WMO_WIND','WMO_PRES','WMO_AGENCY','TRACK_TYPE','DIST2LAND','LANDFALL','IFLAG'],axis=1)

In [4]:
# Drop all data points prior to year=1900
simple_df = simple_df.iloc[9461:]

In [5]:
# Convert ISO_TIME to datetime
simple_df['ISO_TIME'] = pd.to_datetime(simple_df['ISO_TIME'], format='%d/%m/%Y %H:%M')

In [6]:
# Remove all data points which are not in 3 hour increments 
simple_df = simple_df[simple_df['ISO_TIME'].dt.hour % 3 == 0]

In [7]:
# Convert longitudes and latitudes to x, y, z coordinates
def FeatureColumnsXYZ(dframe):
    dframe['x'] = np.cos(np.radians(dframe.LAT)) * np.cos(np.radians(dframe.LON))
    dframe['y'] = np.cos(np.radians(dframe.LAT)) * np.sin(np.radians(dframe.LON))
    dframe['z'] = np.sin(np.radians(dframe.LAT))

In [8]:
FeatureColumnsXYZ(simple_df)

We must now remove all hurricanes in the dataset with too few datapoints (<15)

In [9]:
SID_values = []
for value in simple_df['SID']:
    if value not in SID_values:
        SID_values.append(value)

In [10]:
grouped = simple_df.groupby('SID')

In [11]:
# remove all hurricanes with less than 15 data points approx ~200
SID_to_drop = []
for value in SID_values:
    if len(grouped.groups[value]) < 15:
        if value not in SID_to_drop:
            SID_to_drop.append(value)

In [12]:
def drop_rows_by_sid(df, sid_list):
    # Drop rows where the SID is in the sid_list
    df_filtered = df[~df['SID'].isin(sid_list)]
    
    return df_filtered
simple_df = drop_rows_by_sid(simple_df,SID_to_drop)

In [13]:
grouped = simple_df.groupby('SID')

In [14]:
countSID = []
for value in simple_df['SID']:
    if value not in countSID:
        countSID.append(value)

In [15]:
# Confirm number of removed hurricanes
print("original: " + str(len(SID_values)))
print("new: " + str(len(countSID))) 

original: 3911
new: 3649


## Building and Fine Tuning the Model

In [16]:
def create_sequences(df, window_size):
    sequences = []
    targets = []
    grouped = df.groupby('SID')
    
    for _, group in grouped:
        for i in range(len(group) - window_size):
            seq = group[['x', 'y', 'z']].iloc[i:i+window_size].values
            target = group[['x', 'y', 'z']].iloc[i+window_size].values
            sequences.append(seq)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [17]:
window_size = 8 # 24 hours
X, y = create_sequences(simple_df, window_size)

In [20]:
# Split the data into training and evaluation
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2)

In [21]:
# Understand data shape
print(X_train.shape)
print(y_train.shape)
print(X_eval.shape)
print(y_eval.shape)

(155897, 8, 3)
(155897, 3)
(38975, 8, 3)
(38975, 3)


In [22]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=(8, 3)))
    model.add(
        layers.LSTM(
            # Tune number of units.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            # Tune the activation function to use.
            activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )
    # Tune whether to use dropout.
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(3, activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [23]:
build_model(keras_tuner.HyperParameters())

<Sequential name=sequential, built=True>

In [24]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="cur_dir",
    project_name="hurricane_RNN",
)

In [25]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [26]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_eval, y_eval))

Trial 3 Complete [00h 09m 27s]
val_accuracy: 0.8180115222930908

Best val_accuracy So Far: 0.8180115222930908
Total elapsed time: 00h 27m 19s
